In [1]:
import pandas as pd

df = pd.read_csv("../data/data.csv")

In [2]:
df.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,...,cal_from_macros,pct_carbs,protein_per_kg,pct_HRR,pct_maxHR,cal_balance,lean_mass_kg,expected_burn,Burns Calories (per 30 min)_bc,Burns_Calories_Bin
0,34.91,Male,65.27,1.62,188.58,157.65,69.05,1.00,1080.90,Strength,...,2139.59,0.500432,1.624789,0.741237,0.835985,725.10,47.777394,685.1600,7.260425e+19,Medium
1,23.37,Female,56.41,1.55,179.43,131.75,73.18,1.37,1809.91,HIIT,...,1711.65,0.500850,1.514093,0.551247,0.734270,-232.91,40.809803,978.6184,1.020506e+20,High
2,33.20,Female,58.98,1.67,175.04,123.95,54.96,0.91,802.26,Cardio,...,1965.92,0.500610,1.663445,0.574534,0.708124,805.74,44.635580,654.5266,1.079607e+20,High
3,38.69,Female,93.78,1.70,191.21,155.10,50.07,1.10,1450.79,HIIT,...,1627.28,0.499533,0.862017,0.744155,0.811150,1206.21,63.007432,773.6300,8.987921e+19,High
4,45.09,Male,52.42,1.88,193.58,152.88,70.84,1.08,1166.40,Strength,...,2659.23,0.500581,2.538153,0.668405,0.789751,303.60,43.347504,711.4176,5.264685e+19,Low


In [3]:
df.isnull().sum().sum() 

np.int64(0)

--> Không có mẫu nào bị thiếu

### 1. Loại bỏ các cột không được chú thích trong phần giới thiệu của data trên Kaggle

In [4]:
columns = [
    "Age",
    "Gender",
    "Weight (kg)",
    "Height (m)",
    "Max_BPM",
    "Avg_BPM",
    "Resting_BPM",
    "Session_Duration (hours)",
    "Calories_Burned",
    "Workout_Type",
    "Fat_Percentage",
    "Water_Intake (liters)",
    "Workout_Frequency (days/week)",
    "Experience_Level",
    "BMI",
    "Daily meals frequency",
    "Physical exercise",
    "Carbs",
    "Proteins",
    "Fats",
    "Calories",
    "meal_name",
    "meal_type",
    "diet_type",
    "sugar_g",
    "sodium_mg",
    "cholesterol_mg",
    "serving_size_g",
    "cooking_method",
    "prep_time_min",
    "cook_time_min",
    "rating",
    "Name of Exercise",
    "Sets",
    "Reps",
    "Benefit",
    "Burns Calories (per 30 min)",
    "Target Muscle Group",
    "Equipment Needed",
    "Difficulty Level",
    "Body Part",
    "Type of Muscle",
    "Workout"
]

print(len(columns))

df = df[columns]

43


### 3. Chọn biến mục tiêu là biến Fat_Percentage --> Chọn ra các cột số và loại bỏ ngoại lai của các cột đó --> tính corelations để loại bỏ bớt các biến có độ tương quan thấp

In [5]:
df_number = df.select_dtypes(exclude='object')

Loại bỏ các ngoại lai

In [6]:
def remove_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1

    top = Q3 + 1.5 * IQR
    bottom = Q1 - 1.5 * IQR

    col_new = col.where((col >= bottom) & (col <= top))
    return col_new

df[df_number.columns] = df_number.apply(remove_outliers)

Khi apply hàm remove_outliers lên các cột trong X thì các mẫu ngoại lai sẽ bị thay bằng Nan, có khoảng 8940 mẫu Nan

In [7]:
df.isnull().sum().sum()

np.int64(8129)

Drop các mẫu có giá trị Nan

In [8]:
df = df.dropna()

In [9]:
X = df.drop(columns='Fat_Percentage')
X_corr = X.select_dtypes(exclude="object")
y = df['Fat_Percentage']

Tính độ tương quan

In [10]:
from scipy.stats import pearsonr, spearmanr

for i, x in enumerate(X_corr.columns):
    r, p = pearsonr(X[x], y)
    print(f"{i + 1} . {x} (r : {r}, p : {p})")
    # sp, p = spearmanr(X[x], y)

1 . Age (r : -0.012219287086894196, p : 0.1514204237413938)
2 . Weight (kg) (r : 0.7613700659863277, p : 0.0)
3 . Height (m) (r : -0.13718077812372845, p : 6.885453363530655e-59)
4 . Max_BPM (r : 0.11398412837804345, p : 4.3218481692802245e-41)
5 . Avg_BPM (r : -0.0205122425805868, p : 0.01602785861832214)
6 . Resting_BPM (r : -0.02396548511361602, p : 0.004895610878679386)
7 . Session_Duration (hours) (r : -0.030634010559030315, p : 0.0003217692109785858)
8 . Calories_Burned (r : -0.02640804751198949, p : 0.00193061324543039)
9 . Water_Intake (liters) (r : 0.18774475216590827, p : 1.465776919205597e-109)
10 . Workout_Frequency (days/week) (r : -0.058138051288422136, p : 8.439936175606545e-12)
11 . Experience_Level (r : -0.04074556572911093, p : 1.707339290088298e-06)
12 . BMI (r : 0.9123407929789366, p : 0.0)
13 . Daily meals frequency (r : 0.025013229251022534, p : 0.0033153413454010024)
14 . Physical exercise (r : 0.003189669288767389, p : 0.7080687980957514)
15 . Carbs (r : 0.01275